In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su responsabilidad es leer un archivo CSV de novedades pre-procesado
# y fusionarlo (MERGE) con la tabla de dimensiones 'category'.
from pyspark.sql.functions import col

# 1. --- Configuración ---
# La ruta al archivo CSV de novedades que fue subido por tu script local.
# Asegúrate de que el nombre del archivo coincida con el que se subió.
updates_csv_path = "/Volumes/workspace/tecnomundo_data_raw/uploads_dimensions/Nuevos_productos_dimensions.csv"

# La tabla de dimensiones principal que vamos a actualizar
target_dimension_table = "workspace.tecnomundo_data_dimensions.category"

print(f"Archivo de novedades (CSV): {updates_csv_path}")
print(f"Tabla de destino a actualizar: {target_dimension_table}")


# 2. --- Lectura de las Novedades ---
# Leemos el archivo CSV. Como ya fue pre-procesado localmente,
# los nombres de las columnas y los datos ya están estandarizados.
print("\nLeyendo archivo CSV de novedades...")
try:
    df_updates = (spark.read
                  .format("csv")
                  .option("header", "true")
                  .load(updates_csv_path))

    # Creamos una vista temporal para usarla en el comando MERGE
    df_updates.createOrReplaceTempView("category_updates_temp_view")
    print("Novedades leídas y listas para la fusión.")

except Exception as e:
    print(f"Error al leer el archivo CSV desde el Volume: {e}")
    dbutils.notebook.exit("No se pudo leer el archivo de novedades. Verifica la ruta y que el archivo exista.")


# 3. --- Fusión de Datos con MERGE INTO ---
# Este es el comando clave. Compara la tabla de destino con la vista temporal
# y aplica las reglas de inserción o actualización.
print(f"\nEjecutando MERGE en la tabla '{target_dimension_table}'...")

try:
    spark.sql(f"""
      MERGE INTO {target_dimension_table} AS target
      USING category_updates_temp_view AS source
      ON target.codigo_producto = source.codigo_producto
      
      -- Si el producto ya existe, actualizamos su nombre y categoría
      WHEN MATCHED THEN
        UPDATE SET
          target.nombre_del_producto = source.nombre_del_producto,
          target.categoria = source.categoria
          
      -- Si el producto es nuevo, lo insertamos
      WHEN NOT MATCHED THEN
        INSERT (codigo_producto, nombre_del_producto, categoria)
        VALUES (source.codigo_producto, source.nombre_del_producto, source.categoria)
    """)

    print("¡Éxito! La tabla de dimensiones ha sido actualizada.")

except Exception as e:
    print(f"Ocurrió un error durante la operación MERGE: {e}")


# 4. --- Verificación Final ---
print("\nMostrando una muestra de la tabla de dimensiones actualizada (ordenada por código descendente):")
display(spark.table(target_dimension_table).orderBy(col("codigo_producto").desc()))

